In [1]:
Training_data=[['Green',3,'Mango'],
                ['Yellow',3,'Mango'],
                ['Red',1,'Grape'],
                ['Red',1,'Grape'],
                ['Yellow',3,'Lemon']
               ]
#Column labels
header = ["color","diameter","label"]

def unique_vals(rows,col):
    """Find the unique values for column in a data set"""
    return set([row[col] for row in rows])
#Demo
#unnique_vals(Trainind_data,0) then it will return the set of unique items in 1st coulmn
def class_counts(rows):
    """count the each type of example in a data set"""
    counts={}# a dictionary 
    for row in rows:
        #in our data set the last column is always a Label
        label=row[-1]
        if label not in counts:
            counts[label]=0
        counts[label]+=1
    return counts
#Demo
#class_count(Training_data)

def is_numeric(value):
    """Test if a alue is numeric"""
    return isinstance(value,int) or isinstance(value,float)
#demo
#is_numberic(7)

class Question:
    """A question is used to partition a data set.
    this class just record the column number and a
    column value (e.g. green). The match method is used to compare the features in an
    example to the feature values stored in the question."""
    def __init__(self,column,value):
        self.column=column
        self.value=value
    def match(self,example):
        #compare the feature values in the example to the feature values in this question
        val=example[self.column]
        if is_numeric(val):
            return val>=self.value
        else:
            return val == self.value
    def __repr__(self):
        # this is just a helper method to print the question in a readable format
        condition= '=='
        if is_numeric(self.value):
            condition = ">="
        return "Is %s%s%s" %( header[self.column],condition,str(self.value))
        
def partition(rows,question):
    """Partition a data set
    For each row in the data set, check if it match the question, If so
    , add it to 'true rows' , otehr wise add it to 'false row'"""
    true_rows,false_rows=[],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
        return true_rows, false_rows
    
#Demo
# let's partition the training data based on whether rows are red.
#true_rows,false_rows- partition(training_data,Question(0,'Red'))
#this will contain the all Red rows assign to #true_rows
#this will contain eerythig false assign to #false_rows


def gini(rows):
    """calculate the gini impurity for the list of rows. """
    counts=class_counts(rows)
    impurity=1
    for lbl in counts:
        prob_of_lbl=counts[lbl]/float(len(rows))
        impurity-=prob_of_lbl**2
    return impurity
def info_gain(left,right,current_uncertainty):
    """ Information gain , the uncertainity of the starting node, minus the 
    weighted impurity of two child nodes"""
    p=float(len(left))/(len(left)+len(right))
    return current_uncertainity-p*gini(left)-(1-p)*ginni(right)

def find_best_split(rows):
    """find the best question to ask by iterating over every feature/ value
    and calculate the information gain"""
    best_gain=0 # keep track of the info gain
    best_question = None # keep train  of the feature / value that produced it
    current_uncertainity=gini(rows)
    n_features=len(rows[0])-1 # number of columns
    
    for col in range(n_features): # for each feature
        values=set([row[col] for row in rows])# unique values in the column
        
        for val in values: # for each alues
            question= Question(col,val)
            # try spliting the data set
            true_rows,false_rows=partition(rows,question)
            # skip this split if it doesn't deide the data set
            if len(true_rows)==0 or len(false_rows)==0:
                continue
            # calculate the information gainn form the split
            gain= info_gain(true_rows,false_rows,current_unncertainity)
            
            if gain>= best_gain:
                best_gain,best_question=gain,question
            
        return best_gain, best_question
        
class leaf:
    """A leaf node classifies data.
    this holds a dictionary of class (e.g. "mangp") -> number of times
    it appears in the rows from the training data that reach this leaf"""
            
    def __init__(self,rows):
        self.predictions=class_counts(rows)

class Decision_node:
    """ A decision node ask a question, this holds a reference to the question, and to the two child nodes"""
    def __init__(self, question, true_branch, false_branch):
        self.question=question
        self.true_branch=true_branch
        self.false_branch=false_branch
        

def build_tree(rows):
    # try partitioing the dataset on each of the unnique attribute,
    #calculate the information gain
    #and return the question that produces the highest gain.
    gain, question=find_best_split(rows)
    
    
    #Base case: no further info gain
    #since we can ask no further questiomns
    # we'll return a leaf.
    if gain==0:
        return leaf(rows)
    
    #if we reach here, we have found a useful feature/ value
    #to partition on
    true_rows, false_rows=partition(rows,question)
    
    #Recursivily built the true branch
    true_branch =build_tree(true_rows)
    
    #Recursivily built the false branch
    false_branch =build_tree(false_rows)
    
    # Return a question node, this records the best feature/value to ask at this point,
    #as well as the branches to follow depending on the answer.
    return Decision_Node(question,true_branch,false_brach)

def print_tree(node,spacing=""):
    """World most elegent tree printing function """
    # base case ; we have reached a leaf
    if isinstance(node,leaf):
        print(spacing +"Predict",node.predictions)
        return
    #print the question at this node
    print(spacing + str(node.question))
    
    #call this function recursivly on the true branch
    print(spacing+"-->True:")
    print_tree(node.true_branch,spacing+" ")
    
    #call this funnction recursivvly on the false branch
    print(spacing+"-->False:")
    print_tree(node.false_branch,spacing+" ")
    
def classify(row,node):
    #data case: we've reached a leaf
    if isinstance(node,leaf):
        return node.predictions
    #Decide whether to follow the true branch or false branch.
    #compare the feature / alue stared inn the node,
    # to the example we've considering
    
    if node.question.match(row):
        return classify(row,node.true_branch)
    else:
        return classify(row,node.false_branch)
    
def print_leaf(counts):
    """print the predicitons at a leaf"""
    total= sum(counts.values())*1.0
    probs={}
    for lbl in counts.keys():
        probs[lbl]=str(int(counts[lbl]/total*100)) + "%"
        return probs
    
#######
#Demo:
#printing that a bit nicer
#print_leaf(classify(trainning_data[0],my_tree))
#########
#######
#Demo:
#on the second exmple the confidence is lower
#print_leaf(classify(trainning_data[1],my_tree))
#########

if name == '__main__':
    my_tree=build_tree(training_data)
    print_tree(my_tree)
    
    #Evaluate
    training_data=[['Green',3,'Mango'],
                ['Yellow',4,'Mango'],
                ['Red',2,'Grape'],
                ['Red',1,'Grape'],
                ['Yellow',3,'Lemon']
               ]
    for row in testing_data:
        print("Actual : %s, Predicited : %s"% (row[-1], print_leaf(classify(row,my_tree))))
        

NameError: name 'name' is not defined